# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [16]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
 
    #TODO 1: Add your code here
    df_eng = df_eng.rename(columns={'Movie / TV Series':'Title', 
                                'Year':'Year', 
                                'Synopsis':'Synopsis', 
                                'Review':'Review'})
    df_eng["Original Language"]= 'eng'
    
    
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr = df_fr.rename(columns={'Titre':'Title', 
                                'Année':'Year', 
                                'Synopsis':'Synopsis', 
                                'Critiques':'Review'})
    df_fr["Original Language"]= 'fr'
    
    df = pd.concat([df_eng, df_fr] , ignore_index=True)
    
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp = df_sp.rename(columns={'Título':'Title', 
                                'Año':'Year', 
                                'Sinopsis':'Synopsis', 
                                'Críticas':'Review'})
    df_sp["Original Language"]= 'sp'
    
    df = pd.concat([df, df_sp] , ignore_index=True)

    return df

df = preprocess_data()

In [17]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
12,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",fr
27,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",sp
23,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",sp
20,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",sp
14,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",fr
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",eng
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",fr
17,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",fr
26,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",sp
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",eng


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [22]:
from transformers import MarianMTModel, MarianTokenizer
 

In [27]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = 'Helsinki-NLP/opus-mt-fr-en' #None
es_en_model_name = 'Helsinki-NLP/opus-mt-es-en' #None

fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name) #None
es_en_model = MarianMTModel.from_pretrained(es_en_model_name) #None

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name) #None
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name) #None

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt") #None

    # generate the translation using the model
    outputs = model.generate(**inputs) #None

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [78]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = None
fr_reviews_en =  df[df['Original Language']
                    == 'fr']['Review'].apply(lambda x: translate(x,  
                                                                 fr_en_model, 
                                                                 fr_en_tokenizer ))#None

df.loc[df['Original Language'] == 'fr', ['Review']] = fr_reviews_en 
 

# filter synopsis in French and translate to English
fr_synopsis = None
fr_synopsis_en = df[df['Original Language']
                    == 'fr']['Synopsis'].apply(lambda x: translate(x,  
                                                                 fr_en_model, 
                                                                 fr_en_tokenizer ))

df.loc[df['Original Language'] == 'fr', ['Synopsis']] = fr_synopsis_en 

# filter reviews in Spanish and translate to English
es_reviews = None
es_reviews_en = df[df['Original Language']
                    == 'sp']['Review'].apply(lambda x: translate(x,  
                                                                 es_en_model, 
                                                                 es_en_tokenizer ))#None

df.loc[df['Original Language'] == 'sp', ['Review']] = es_reviews_en 

# filter synopsis in Spanish and translate to English
es_synopsis = None
es_synopsis_en = df[df['Original Language']
                    == 'sp']['Synopsis'].apply(lambda x: translate(x,  
                                                                 es_en_model, 
                                                                 es_en_tokenizer ))#None

df.loc[df['Original Language'] == 'sp', ['Synopsis']] = es_synopsis_en 

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

In [79]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",eng
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",fr
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",sp
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",fr
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",fr
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",eng
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",sp
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",sp


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [ ]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = None
sentiment_classifier = None

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    return None

In [ ]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column

In [ ]:
df.sample(10)

In [ ]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)